## Multivariate Time Series Prediction Using LSTM
In this example, we will implement an LSTM using mlpack to forecast multivariate time series data.
This notebook contains a step by step walkthrough for time series prediction using LSTMs.

### 1. Setup
Include all libraries required to implement this tutorial. These mainly include files from mlpack, ensmallen and armadillo.

In [1]:
#include <mlpack/core.hpp>
#include <mlpack/methods/ann/rnn.hpp>
#include <mlpack/methods/ann/layer/layer.hpp>
#include <mlpack/core/data/scaler_methods/min_max_scaler.hpp>
#include <mlpack/methods/ann/init_rules/he_init.hpp>
#include <mlpack/methods/ann/loss_functions/mean_squared_error.hpp>
#include <ensmallen.hpp>

Some convenient namespaces to simplify the tutorial.

In [2]:
using namespace mlpack;

In [3]:
using namespace mlpack::ann;

In [4]:
using namespace arma;

In [5]:
using namespace std;

### 2. Set Model and Training parameters.
Set the training parameters for the model.
Parameters include the step size for the optimizer, ratio for the train/test split, batch size for training, the look-back parameter for LSTM, and number of hidden cells.

In [6]:
// Testing data is taken from the dataset in this ratio.
const double RATIO = 0.1;

// Step size of an optimizer.
const double STEP_SIZE = 5e-4;

// Number of cells in the LSTM (hidden layers in standard terms).
// NOTE: you may play with this variable in order to further optimize the
// model (as more cells are added, accuracy is likely to go up, but training
// time may take longer).
const int H1 = 16;

// Number of data points in each iteration of SGD.
const size_t BATCH_SIZE = 16;

// Nunmber of timesteps to look backward for in the RNN.
const int rho = 16;

// Max Rho for LSTM.
const int maxRho = rho;

// Number of epochs for training. Increase the epochs for better results.
const int EPOCHS = 15;

Set paths for the dataset, trained model and final predictions.

In [7]:
// Change the names of these files as necessary. They should be correct
// already, if your program's working directory contains the data and/or
// model.
const std::string dataFile = "Google2016-2019.csv";
// Path where the model will be saved.
const std::string modelFile = "lstm_multi.bin";

### 3. Loading the Dataset.
Dataset will be loaded and preprocessed for the model. If we want to predict the Google stock price correctly then we need to consider the volume of the stocks traded, the closing, opening, high and low values of the stock price from the previous days. This is a time series problem. We will create data for the training of the RNN model that will go back 25 business days in the past for each time step. We will convert the input data to the time series format the RNN requires. We will take 30 % of the latest data as our test dataset.

In [8]:
arma::mat dataset;

// Armadillo is column-major, so each column represents one data point
// and each row represents one dimension.
mlpack::data::Load(dataFile, dataset, true);
// Visualize the first the rows of the dataset.
dataset.submat(1, 0, dataset.n_rows - 1, 3).t().print()
// Here each row represents: close, volume, open, high, low.
// The 0s in the first row represent the column headers.
// We have printed the transposed dataset since
// Armadillo represents a data in a column-major format,
// but we want to print in a row-major format (one data point per row).

            0            0            0            0            0
   6.6826e+02   2.6320e+06   6.7100e+02   6.7230e+02   6.6328e+02
   6.8004e+02   2.1697e+06   6.7897e+02   6.8033e+02   6.7300e+02
   6.8411e+02   1.9314e+06   6.8300e+02   6.8743e+02   6.8141e+02


### 4. Preprocess the dataset.
Since columns such as date and time are not needed we will drop them. We will also scale the data to increase stability in training.

In [9]:
// The CSV file has a header, so it is necessary to remove it. In Armadillo's
// representation it is the first column.
// The first column in the CSV is the date which is not required, therefore
// we remove it also (first row in in arma::mat).

dataset = dataset.submat(1, 1, dataset.n_rows - 1, dataset.n_cols - 1);

// We have 5 input data columns and 2 output columns (target).
size_t inputSize = 5, outputSize = 2;

// Split the dataset into training and validation sets.
arma::mat trainData;
arma::mat testData;
data::Split(dataset, trainData, testData, RATIO, false);

In [10]:
mlpack::data::MinMaxScaler scale;
// Fit scaler only on training data.
scale.Fit(trainData);
scale.Transform(trainData, trainData);
scale.Transform(testData, testData);

### 5. Create Time Series Dataset.
The time series data for training the model contains the Closing stock price, the Volume of stocks traded, Opening stock price, Highest and Lowest stock price for 'rho' days in the past.
The two target variables (multivariate) we want to predict are the Highest stock price and Lowest stock price (high, low) for the next day! This function essentially reshapes the dataset into desired shape for LSTMs. Here the dataset will be converted to a cube having rho slices where each slice contains values of input features for that day. Similarly the prediction data i.e. high and low of stock market for corresponding input features are stacked behind one another for rho days.

In [11]:
template<typename InputDataType = arma::mat,
         typename DataType = arma::cube,
         typename LabelType = arma::cube>
void CreateTimeSeriesData(InputDataType dataset,
                          DataType& X,
                          LabelType& y,
                          const size_t rho,
                          const size_t inputSize = 5,
                          const size_t outputSize = 2)
{
  X.set_size(inputSize, dataset.n_cols - rho + 1, rho);
  y.set_size(outputSize, dataset.n_cols - rho + 1, rho);
  for (size_t i = 0; i < dataset.n_cols - rho; i++)
  {
    X.subcube(arma::span(), arma::span(i), arma::span()) =
        dataset.submat(arma::span(), arma::span(i, i + rho - 1));
    y.subcube(arma::span(), arma::span(i), arma::span()) =
        dataset.submat(arma::span(3, 4), arma::span(i + 1, i + rho));
  }
}

In [12]:
// We need to individually create training and testing time series data.
arma::cube trainX, trainY, testX, testY;
// Create training sets for one-step-ahead regression.
CreateTimeSeriesData(trainData, trainX, trainY, rho);
// Create test sets for one-step-ahead regression.
CreateTimeSeriesData(testData, testX, testY, rho);

### 6. Create the Model
We add 3 LSTM modules that will be stacked one after the other in the RNN, implementing an efficient stacked RNN. Finally, the output will have 2 units the (high, low) values of the stock price for the next day.

In [13]:
mlpack::ann::RNN<mlpack::ann::MeanSquaredError<>, mlpack::ann::HeInitialization> model(rho);
// Create the model architecture.
model.Add<mlpack::ann::IdentityLayer<> >();
model.Add<mlpack::ann::LSTM<> >(inputSize, H1, maxRho);
model.Add<mlpack::ann::Dropout<> >(0.6);
model.Add<mlpack::ann::LeakyReLU<> >();
model.Add<mlpack::ann::LSTM<> >(H1, H1, maxRho);
model.Add<mlpack::ann::Dropout<> >(0.6);
model.Add<mlpack::ann::LeakyReLU<> >();
model.Add<mlpack::ann::LSTM<> >(H1, H1, maxRho);
model.Add<mlpack::ann::ReLULayer<> >();
model.Add<mlpack::ann::Linear<> >(H1, outputSize);

### 7. Training the model.
We will use ensmallen to create the optimizer and train the model. For more details refer to the [documentation](https://www.ensmallen.org/docs.html).

In [14]:
ens::Adam optimizer(
    STEP_SIZE, // Step size of the optimizer.
    BATCH_SIZE, // Batch size. Number of data points that are used in each iteration.
    0.9, // Exponential decay rate for the first moment estimates.
    0.999, // Exponential decay rate for the weighted infinity norm estimates.
    1e-8, // Value used to initialise the mean squared gradient parameter.
    trainData.n_cols * EPOCHS, // Max number of iterations.
    1e-8,// Tolerance.
    true);

optimizer.Tolerance() = -1;

Here we will use ensmallen callbacks to train the model. We will be using Adam optimizer. We will terminate optimization when the loss stops decreasing or doesn't show any improvement.

In [15]:
// Train the model.
model.Train(trainX,
            trainY,
            optimizer,
            // PrintLoss Callback prints loss for each epoch.
            ens::PrintLoss(),
            // Progressbar Callback prints progress bar for each epoch.
            // Here 40 signifies width of progress bar.
            ens::ProgressBar(40),
            // Stops the optimization process if the loss stops decreasing
            // or no improvement has been made. This will terminate the
            // optimization once we obtain a minima on training set.
            ens::EarlyStopAtMinLoss());

Epoch 1/244
0.502446
42/42 [==================================================] 100% - 0s 9ms/step - loss: 0.502446
Epoch 2/244
0.430152
42/42 [==================================================] 100% - 0s 10ms/step - loss: 0.430152
Epoch 3/244
0.207513
42/42 [==================================================] 100% - 0s 10ms/step - loss: 0.207513
Epoch 4/244
0.178334
42/42 [==================================================] 100% - 0s 10ms/step - loss: 0.178334
Epoch 5/244
0.162722
42/42 [==================================================] 100% - 0s 10ms/step - loss: 0.162722
Epoch 6/244
0.145162
42/42 [==================================================] 100% - 0s 10ms/step - loss: 0.145162
Epoch 7/244
0.127502
42/42 [==================================================] 100% - 0s 9ms/step - loss: 0.127502
Epoch 8/244
0.11174
42/42 [==================================================] 100% - 0s 10ms/step - loss: 0.11174
Epoch 9/244
0.0978461
42/42 [=======================================

### 8. Running inference and printing the results
Since the predicted is in form of a cube we need to convert it to matrix with predicted values. Then we take the inverse transform to convert it to meaningful data. The prediction results are the (high, low) for the next day and come from the last slice of the prediction. The last 2 columns are the predictions, the preceding columns are the data used to generate those predictions.

In [16]:
// Run Inference.
arma::cube predOutP;
model.Predict(testX, predOutP);

In [19]:
arma::mat flatDataAndPreds = testX.slice(testX.n_slices - 1);
// The prediction results are the (high, low) for the next day and come from
// the last slice from the prediction.
flatDataAndPreds.rows(flatDataAndPreds.n_rows - 2,
    flatDataAndPreds.n_rows - 1) = predOutP.slice(predOutP.n_slices - 1);

scale.InverseTransform(flatDataAndPreds, flatDataAndPreds);
// We need to remove the last column because it was not used for training
// (there is no next day to predict).
flatDataAndPreds.shed_col(flatDataAndPreds.n_cols - 1);

// NOTE: we do not have the last data point in the input for the prediction
// because we did not use it for the training, therefore the prediction result
// will be for the day before. In your own application you may of course load
// any dataset for prediction.
std::cout << "The predicted Google stock (high, low) for the last day is: " << std::endl
    << " (" << flatDataAndPreds(flatDataAndPreds.n_rows - 2, flatDataAndPreds.n_cols - 1) << ", "
    << flatDataAndPreds(flatDataAndPreds.n_rows - 1, flatDataAndPreds.n_cols - 1) << ")"
    << std::endl;

The predicted Google stock (high, low) for the last day is: 
 (1108.29, 1029.65)


### 9. Loading and Saving Models
In the real world, we won't be training the model from scratch everytime we need to run inference.
We will save the model once and load it as many times as we want for either training or inference.

In [20]:
// Save the model.
mlpack::data::Save(modelFile, "LSTMMulti", model);
// Load the model.
RNN<mlpack::ann::MeanSquaredError<>, mlpack::ann::HeInitialization> trainedModel(rho);
mlpack::data::Load(modelFile, "LSTMMulti", trainedModel);